In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from google.colab import files


In [0]:
data_path = os.path.join("/content/gdrive/My Drive/", "DRU-MAWI-project/ICHI14_dataset/data")
patient_list = ['002','003','005','007','08a','08b','09a','09b', '10a','011','013','014','15a','15b','016',
            '017','018','019','020','021','022','023','025','026','027','028','029','030','031','032',
            '033','034','035','036','037','038','040','042','043','044','045','047','048','049','051']

In [0]:
train_patient_list, test_patient_list = train_test_split(patient_list, random_state=152, test_size=0.3)
test_patient_list, valid_patient_list = train_test_split(test_patient_list, random_state=151, test_size=0.5)

In [5]:
print(len(patient_list))
print(len(train_patient_list))
print(len(valid_patient_list))
print(len(test_patient_list))

45
31
7
7


In [6]:
print(train_patient_list)
print(valid_patient_list)
print(test_patient_list)

['022', '09b', '048', '020', '023', '15b', '003', '042', '15a', '038', '025', '011', '018', '029', '031', '014', '08a', '047', '049', '016', '040', '005', '037', '033', '013', '017', '026', '044', '007', '027', '10a']
['035', '034', '051', '019', '045', '043', '08b']
['021', '09a', '002', '028', '032', '036', '030']


In [0]:
def change_labels(sample):
    """
    Returns:
    sample - contains only label 1(awake) and 0(sleep) for polisomnography
    """
    
    sample.gt[sample.gt==0] = 8
    sample.gt[np.logical_or.reduce((sample.gt==1, sample.gt==2, sample.gt==3, sample.gt==5))] = 0
    sample.gt[np.logical_or.reduce((sample.gt==6, sample.gt==7, sample.gt==8))] = 1
    
    return sample   

#-------------------------------------------------------------------------

def decoder(sample):
    '''
    Returns: 
    decoded_sample - contains accelerometer and ps data for each sensor record, ndarray of shape (n_records, 4)
    
    '''

    sample = np.repeat(sample, sample.d, axis=0)
    n_records = sample.shape[0]
    decoded_sample = np.zeros((n_records, 4))
    
    decoded_sample[:, 0] = sample.x
    decoded_sample[:, 1] = sample.y
    decoded_sample[:, 2] = sample.z
    decoded_sample[:, 3] = sample.gt
    
    return decoded_sample

#-------------------------------------------------------------------------

def divide_by_windows(decoded_sample, window_len=60):
    """
    Parameters:
    wondow_len - length of each window in seconds, int
    Returns:
    X - accelerometer data, ndarray of shape (n_windows, window_len, 3)
    y - polisomnography data, ndarray of shape (n_windows, )
    """
    
    window_len *= 100
    n_windows = decoded_sample.shape[0] // window_len
    
    X = np.zeros((n_windows, window_len, 3))
    y = np.zeros(n_windows)
    
    for i in range(n_windows):
        X[i] = decoded_sample[window_len * i: window_len * i + window_len, 0: 3]
        
        ones = np.count_nonzero(decoded_sample[window_len*i: window_len*i+window_len, 3])
        if ones >= (window_len / 2):
            y[i] = 1
        else:
            y[i] = 0
                
    return X, y

#-------------------------------------------------------------------------

def get_one_patient_data(data_path, patient, window_len=60):
    
    """
    Returns:
    X, y - for one patient
    """
    
    sample = np.load("%s/p%s.npy"%(data_path, patient)).view(np.recarray)
    sample = change_labels(sample)
    sample = decoder(sample)
    X, y = divide_by_windows(sample, window_len)
    
    return X, y

#-------------------------------------------------------------------------

def get_data_for_model(data_path, patient_list, window_len=60):
    
    """
    Returns:
    X, y - for all patient list, ndarray of shape (n_records, n_features, n_channels=3)
    """
    
    X_all_data = []
    y_all_data = []
    for patient in patient_list:
        X, y = get_one_patient_data(data_path, patient, window_len)
        X_all_data.append(X)
        y_all_data.append(y)
        
    X_all_data = np.concatenate(X_all_data, axis=0)
    y_all_data = np.concatenate(y_all_data, axis=0)
    
    return X_all_data, y_all_data
  
#-------------------------------------------------------------------------

def get_dawnsampled_data(data_path, patient_list, window_len=60, dawnsample="pca", n_components=10, n_windows=10):
    
    """
    Parameters:
    dawnsample - "pca", "mean", "max", "mode", None - determine the type of data reducing
    Returns:
    X, y - reduced data for all patient list and combine several windows data, ndarray of shape (n_records, n_components * n_windows, n_channels=3)
    """
    
    X_all_data = []
    y_all_data = []
    for patient in patient_list:
        X, y = get_one_patient_data(data_path, patient, window_len)
        
        if dawnsample.lower() == "pca":
          X = reduce_data_pca(X, n_components=n_components)
          
        elif dawnsample.lower() == "mean":
          X = reduce_data_mean(X, n_components=n_components)
          
        elif dawnsample.lower() == "max":
          X = reduce_data_max(X, n_components=n_components)
          
        elif dawnsample.lower() == "mode":
          X = reduce_data_mode(X, n_components=n_components)
          
        elif dawnsample.lower() == "simple":
          X = reduce_data_simple(X, n_components=n_components)
        
        elif dawnsample.lower() == "statistic":
          X = reduce_data_statistics(X, n_components=n_components)
        
        
        X_new = np.zeros((X.shape[0] - n_windows, X.shape[1] * (n_windows + 1), X.shape[2]))
        
        for i in range(0, X.shape[0] - n_windows):
            X_buff = X[i]
            for j in range(1, n_windows + 1):
                X_buff = np.concatenate([X_buff, X[i+j]], axis=0)
            X_new[i] = X_buff                            
    
    
        if n_windows != 0:
          #y = y[n_windows: ]
          y = y[(n_windows//2): -(n_windows//2)]
      
        
        X_all_data.append(X_new)
        y_all_data.append(y)

        #np.save(("X_p%s.npy"%(patient)), X_new)
        #np.save(("y_p%s.npy"%(patient)), y)
        
    X_all_data = np.concatenate(X_all_data, axis=0)
    y_all_data = np.concatenate(y_all_data, axis=0)
    
    
    
    return X_all_data, y_all_data
  
def reduce_data_pca(X, n_components=300):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_features, n_channels=3)
    """
    pca1 = PCA(n_components)
    pca2 = PCA(n_components)
    pca3 = PCA(n_components)
    
    pca1.fit(X[:, :, 0])
    pca2.fit(X[:, :, 1])
    pca3.fit(X[:, :, 2])
    
    X1 = pca1.transform(X[:, :, 0])
    X2 = pca2.transform(X[:, :, 1])
    X3 = pca3.transform(X[:, :, 2])
    
    X_reduced = np.concatenate([X1, X2, X3], axis=1).reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  
  
def reduce_data_max(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
   
    
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
    
    
    for i in range(n_components):
      
      X_reduced[:, i, :] = np.amax(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
      
    
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  

def reduce_data_mean(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
   
    
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
    
    
    for i in range(n_components):
      
      X_reduced[:, i, :] = np.mean(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
         
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  
    
def reduce_data_mode(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
    
    from scipy.stats import mode
   
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
       
    for i in range(n_components):
      
      X_reduced[:, i, :] = mode(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
         
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced
  
def reduce_data_simple(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
   
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    window_len = X.shape[1] // n_components
       
    for i in range(n_components):
      
      X_reduced[:, i, :] = X[:, i * window_len, :]
         
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced

def reduce_data_statistics(X, n_components=600):
    """
    Parameters:
    X - ndarray of shape (n_samples, n_features)
    
    Returns:
    X, y - reduced data, ndarray of shape (n_records, n_components, n_channels=3)
    """
    
    
   
    X_reduced = np.zeros((X.shape[0], n_components, 3))
    #X_reduced2 = np.zeros((X.shape[0], n_components, 3))
    
    window_len = X.shape[1] // n_components
       
    for i in range(n_components):
      
      #X_reduced[:, i, 0: 3] = X[:, i * window_len, :]
      #X_reduced[:, i, 0: 3] = np.mean(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
      X_reduced[:, i, : ] = np.std(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
      #X_reduced[:, i, 3: 6] = np.ptp(X[:, i * window_len: (i + 1) * window_len, :], axis=1)
      #X_reduced[:, i, 0: 3] = np.sqrt(np.mean(np.square(X[:, i * window_len: (i + 1) * window_len, :]), axis=1))
      #X_reduced2[:, i, : ] = np.sqrt(np.mean(np.square(X[:, i * window_len: (i + 1) * window_len, :]), axis=1))
      #X_reduced[:, i, 0: 3] = np.max(X[:, i * window_len: (i + 1) * window_len, :], axis=1) / X_reduced[:, i, :]
      
        #mean = np.mean(X, axis=1)
        #max_val = np.amax(X, axis=1)
        #min_val = np.amin(X, axis=1)
        #ptp = np.ptp(X, axis=1)
        #rms = np.sqrt(np.mean(np.square(X), axis=1))
        #crest_factor = np.max(X, axis=1) / rms
         
    X_reduced = X_reduced.reshape(X.shape[0], n_components, 3)
    
    return X_reduced

In [0]:
X_train, y_train = get_data_for_model(data_path, train_patient_list, window_len=1)
X_valid, y_valid = get_data_for_model(data_path, valid_patient_list, window_len=1)
X_test, y_test = get_data_for_model(data_path, test_patient_list, window_len=1)

In [10]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(951083, 100, 3)
(214186, 100, 3)
(228481, 100, 3)


In [41]:
%%time
X_train, y_train = get_dawnsampled_data(data_path, train_patient_list, window_len=60, dawnsample="statistic", n_components=10, n_windows=20)
X_valid, y_valid = get_dawnsampled_data(data_path, valid_patient_list, window_len=60, dawnsample="statistic", n_components=10, n_windows=20)
X_test, y_test = get_dawnsampled_data(data_path, test_patient_list, window_len=60, dawnsample="statistic", n_components=10, n_windows=20)

CPU times: user 16.5 s, sys: 563 ms, total: 17.1 s
Wall time: 17.2 s


In [42]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

(15215, 210, 3)
(15215,)
(3425, 210, 3)
(3425,)
(3665, 210, 3)
(3665,)


In [0]:
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from keras.layers import LSTM, Bidirectional

from keras.callbacks import ModelCheckpoint, EarlyStopping


In [11]:
NN = Sequential()

NN.add(Conv1D( 512, 3, input_shape=(6600, 3), activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
#NN.add(Dropout(0.5))
#NN.add(Conv1D( 512, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
#NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=2))
NN.add(Dropout(0.5))

NN.add(Conv1D( 64, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
#NN.add(Dropout(0.5))
#NN.add(Conv1D( 64, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
#NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=2))
NN.add(Dropout(0.5))

NN.add(Conv1D( 32, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
NN.add(BatchNormalization())
#NN.add(Dropout(0.5))
#NN.add(Conv1D( 32, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
#NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=2))
#NN.add(Dropout(0.5))
#NN.add(Flatten())

#NN.add(Dense(64, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
#NN.add(BatchNormalization(axis=1))
#NN.add(Dropout(0.5))

NN.add(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))
#NN.add(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))
NN.add(LSTM(128, dropout=0.15, recurrent_dropout=0.15))
NN.add(Dropout(0.5))
#NN.add(BatchNormalization())


#NN.add(Dense(16, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
#NN.add(BatchNormalization(axis=1))
#NN.add(Dropout(0.2))

NN.add(Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform", kernel_regularizer=l2(0.1)))

NN.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

print(NN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 6598, 512)         5120      
_________________________________________________________________
batch_normalization_1 (Batch (None, 6598, 512)         2048      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3299, 512)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 3299, 512)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3297, 64)          98368     
_________________________________________________________________
batch_normalization_2 (Batch (None, 3297, 64)          256       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1648, 64)          0         
__________

In [45]:
NN = Sequential()

NN.add(Conv1D( 16, 3, input_shape=(210, 3), activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
NN.add(BatchNormalization())
NN.add(Dropout(0.2))
NN.add(Conv1D( 16, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=2))
NN.add(Dropout(0.2))

NN.add(Conv1D( 32, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
NN.add(BatchNormalization())
NN.add(Dropout(0.2))
NN.add(Conv1D( 32, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
NN.add(BatchNormalization())
NN.add(MaxPooling1D( pool_size=2))
NN.add(Dropout(0.2))

NN.add(Conv1D( 64, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
NN.add(BatchNormalization())
NN.add(Dropout(0.5))
NN.add(Conv1D( 64, 3, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.01)))
NN.add(BatchNormalization())
#NN.add(MaxPooling1D( pool_size=2))
#NN.add(Dropout(0.5))
#NN.add(Flatten())

#NN.add(Dense(64, activation="relu", kernel_initializer="he_uniform", kernel_regularizer=l2(0.1)))
#NN.add(BatchNormalization(axis=1))
#NN.add(Dropout(0.5))

NN.add(LSTM(10, dropout=0.1, recurrent_dropout=0.1))
#NN.add(Dropout(0.1))


NN.add(Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform", kernel_regularizer=l2(0.005)))

NN.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

print(NN.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_25 (Conv1D)           (None, 208, 16)           160       
_________________________________________________________________
batch_normalization_25 (Batc (None, 208, 16)           64        
_________________________________________________________________
dropout_27 (Dropout)         (None, 208, 16)           0         
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 206, 16)           784       
_________________________________________________________________
batch_normalization_26 (Batc (None, 206, 16)           64        
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 103, 16)           0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 103, 16)           0         
__________

In [0]:
callbacks = [ModelCheckpoint('CNN_model_raw_data_weights.hdf5', monitor='val_acc', save_best_only=True), EarlyStopping(monitor='val_loss', patience=5)]

In [47]:
%%time

NN.fit(X_train, y_train,
       batch_size=32, 
       epochs=30, 
       validation_data=(X_valid, y_valid), 
       callbacks=callbacks,
       verbose=1)

Train on 15215 samples, validate on 3425 samples
Epoch 1/30
15215/15215 [==============================] - 75s 5ms/step - loss: 2.3079 - acc: 0.7018 - val_loss: 1.2126 - val_acc: 0.6893
Epoch 2/30
15215/15215 [==============================] - 71s 5ms/step - loss: 0.8728 - acc: 0.7408 - val_loss: 0.7721 - val_acc: 0.7355
Epoch 3/30
15215/15215 [==============================] - 71s 5ms/step - loss: 0.6656 - acc: 0.7470 - val_loss: 0.6645 - val_acc: 0.7369
Epoch 4/30
15215/15215 [==============================] - 71s 5ms/step - loss: 0.6019 - acc: 0.7541 - val_loss: 0.9989 - val_acc: 0.5352
Epoch 5/30
15215/15215 [==============================] - 71s 5ms/step - loss: 0.5841 - acc: 0.7520 - val_loss: 0.6916 - val_acc: 0.7168
Epoch 6/30
15215/15215 [==============================] - 71s 5ms/step - loss: 0.5776 - acc: 0.7598 - val_loss: 0.6313 - val_acc: 0.7492
Epoch 7/30
15215/15215 [==============================] - 70s 5ms/step - loss: 0.5726 - acc: 0.7561 - val_loss: 0.6965 - val_acc:

In [48]:
scores = NN.evaluate(X_test, y_test)
print("Test accuracy =", scores[1])

3665/3665 [==============================] - 4s 1ms/step
Test accuracy = 0.7053206002728513


In [0]:
files.download('CNN_model_raw_data_weights.hdf5')

In [0]:
NN.load_weights("CNN_model_raw_data_weights.hdf5")

In [39]:
scores = NN.evaluate(X_test, y_test)
print("Test accuracy =", scores[1])

3749/3749 [==============================] - 4s 1ms/step
Test accuracy = 0.7076553747666045


In [40]:
scores = NN.evaluate(X_valid, y_valid)
print("Valid accuracy =", scores[1])

3509/3509 [==============================] - 3s 990us/step
Valid accuracy = 0.7549159305027388


In [0]:
saved_model = NN.to_json()
with open("CNN_model_raw_data.json", "w") as json_file:
    json_file.write(saved_model)
    
files.download('CNN_model_raw_data.json')

std 60, 12 windows: max test acc = 0.7222, 9 min, 25 epoch, EarlyStopping = 10

pca 30, 16 windows: max test acc = 0.7256, 4 min, 15 epoch, EarlyStopping = 5
